In [ ]:
# default_exp evaluation

# evaluation

> API details.

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#export
import numpy as np
from wxbs_benchmark.dataset import *
from wxbs_benchmark.metrics import *

def evaluate_Fs(Fs = [], subset = 'test'):
    dset = WxBSDataset('.WxBS', subset=subset, download=True)
    ths = np.arange(20)
    gt_corrs = []
    names = []
    for data_dict in dset:
        corrs = data_dict['pts']
        pairname = data_dict['name']
        names.append(pairname)
        gt_corrs.append(corrs)
    assert len(Fs) == len(gt_corrs)
    per_pair_results = {}
    all_res = []
    for (F, pts, pairname) in zip(Fs, gt_corrs, names):
        res = fraction_of_gt_corrs_consisent_with_F(F, pts, ths)
        per_pair_results[pairname] = res
        all_res.append(res)
    per_pair_results['average'] = np.stack(all_res, axis=1).mean(axis=1)
    return per_pair_results, ths

/opt/homebrew/Caskroom/miniforge/base/envs/python39/lib/python3.9/site-packages/kornia/augmentation/augmentation.py:1830: DeprecationWarning: GaussianBlur is no longer maintained and will be removed from the future versions. Please use RandomGaussianBlur instead.
  warnings.warn(


Evaluates fundamental matrices returned by algorithm

In [ ]:
shift_f = np.array([[0.0, 0.0, 0.0],
              [0.0, 0.0, -1.0],
              [0.0, 1.0, 0.0]])
Fs = [shift_f, shift_f]

res_dict, th = evaluate_Fs(Fs, 'val')
print (res_dict)

{'WGABS/petrzin': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.02631579, 0.02631579, 0.02631579, 0.02631579, 0.02631579, 0.05263158, 0.05263158], 'WGALBS/kyiv_dolltheater2': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 'average': array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.01315789, 0.01315789,
       0.01315789, 0.01315789, 0.01315789, 0.02631579, 0.02631579],
      dtype=float32)}


/opt/homebrew/Caskroom/miniforge/base/envs/python39/lib/python3.9/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
def evaluate_corrs(estimated_right = [],
                   estimated_left = [],
                   subset = 'test'):
    dset = WxBSDataset('.WxBS', subset=subset, download=True)
    ths = np.arange(20)
    gt_corrs = []
    names = []
    for data_dict in dset:
        corrs = data_dict['pts']
        pairname = data_dict['name']
        names.append(pairname)
        gt_corrs.append(corrs)
    assert len(estimated_right) == len(gt_corrs)
    assert len(estimated_left) == len(gt_corrs)
    all_res = []
    per_pair_results = {}
    for (est_right, est_left, gt_pts, pairname) in zip(estimated_right,
                                                       estimated_left, 
                                                       gt_corrs, names):
        res = 0.5 * (PCK(est_right, gt_pts[:, 2:4], ths) + 
                     PCK(est_left,  gt_pts[:, :2], ths))
        per_pair_results[pairname] = res
        all_res.append(res)
    per_pair_results['average'] = np.stack(all_res, axis=1).mean(axis=1)
    return per_pair_results, ths

/opt/homebrew/Caskroom/miniforge/base/envs/python39/lib/python3.9/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


We will check the naive baseline -- no camera motion, return the input baseline.

In [ ]:
dset = WxBSDataset('.WxBS', subset='val', download=True)
predicted_left = []
predicted_right = []
for res_dict in dset:
    gt_corrs = res_dict['pts']
    query_left = gt_corrs[:, :2]
    query_right = gt_corrs[:, 2:]
    predicted_right.append(query_left)
    predicted_left.append(query_right)
eval_results = evaluate_corrs (predicted_right, predicted_left, subset='val')
print (eval_results)

({'WGABS/petrzin': array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0.], dtype=float32), 'WGALBS/kyiv_dolltheater2': array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0.], dtype=float32), 'average': array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0.], dtype=float32)}, array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19]))


/opt/homebrew/Caskroom/miniforge/base/envs/python39/lib/python3.9/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
